<a href="https://colab.research.google.com/github/manukj/Machine-Learning/blob/master/Program2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
with open("lab2.csv") as f:
  examples = list(csv.reader(f))
examples = examples[1:]

In [0]:
def g_O(n):
  return ('?',)* n

def s_O(n):
  return ('O',)* n

In [0]:
def more_generalize(h1,h2):
  m_g = []
  for x,y in zip(h1,h2):
    mg = x == '?' or (x != 'O' and (x ==y or y == 'O'))
    m_g.append(mg)
  return all(m_g)

In [0]:
def fullfills(e,h):
  return more_generalize(h,e)

In [0]:
def get_domains(examples):
  domains = [set() for i in examples[0]]
  for x in examples:
    for i,xi in enumerate(x):
      domains[i].add(xi)
  return [list(sorted(x)) for x in domains]

In [0]:
def min_generalize(h,x):
  h_new = list(h)
  for i in range(len(h)):
    if not fullfills(x[i],h[i]):
      h_new[i] = '?' if h[i] != 'O' else x[i]
  return [tuple(h_new)]

In [0]:
def generalize_S(x,G,S):
  S_prev = list(S)
  for s in S_prev:
    if not fullfills(x,s):
      S.remove(s)
      Sminus = min_generalize(s,x)
      S.update([
                h for h in Sminus
                if any(more_generalize(g,h) for g in G)
                ])
      S.difference_update([
                           h for h in S 
                           if any(more_generalize(h,h1) for h1 in S if h1 != h)
      ])
  return S

In [0]:
def min_specialize(h,domain,x):
  results = []
  for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]            
        elif h[i]!= "O":
            h_new = h[:i] + ('O',) + h[i+1:]
        results.append(h_new)
  return results

In [0]:
def specialize_G(x,domains,G,S):
  G_prev = list(G)
  for g in G_prev:
          if fullfills(x, g):
            G.remove(g)
            Gminus = min_specialize(g, domains, x)
                   ## keep only specializations that have a conuterpart in S
            G.update([h for h in Gminus if any([more_generalize(h, s)
                                                for s in S])])
            ## remove hypotheses less general than any other in G
            G.difference_update([h for h in G if
                                 any([more_generalize(g1, h)
                                      for g1 in G if h != g1])])

  return G

In [112]:
def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    G = set([g_O(len(domains))])
    S = set([s_O(len(domains))])
    i = 0
    print("\n G[{0}]:".format(i), G)
    print("\n S[{0}]:".format(i), S)
    for xcx in examples:
        i = i + 1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
        if cx == 'Y':  # x is positive example
            G = {g for g in G if fullfills(x, g)}
            S = generalize_S(x, G, S)
        else:  # x is negative example
            S = {s for s in S if not fullfills(x, s)}
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i), G)
        print("\n S[{0}]:".format(i), S)
    return
candidate_elimination(examples)


 G[0]: {('?', '?', '?', '?', '?')}

 S[0]: {('O', 'O', 'O', 'O', 'O')}

 G[1]: {('?', '?', 'white', '?', '?'), ('?', '?', '?', '?', 'sports'), ('usa', '?', '?', '?', '?'), ('?', 'toyota', '?', '?', '?'), ('?', '?', '?', '1990', '?')}

 S[1]: {('O', 'O', 'O', 'O', 'O')}

 G[2]: {('?', 'toyota', '?', '?', 'economy'), ('?', '?', 'white', '?', '?'), ('?', 'honda', '?', '?', 'sports'), ('usa', '?', '?', '?', '?'), ('?', '?', '?', '1990', '?')}

 S[2]: {('O', 'O', 'O', 'O', 'O')}

 G[3]: {('?', '?', 'white', '?', '?'), ('?', 'honda', '?', '?', 'sports'), ('?', '?', '?', '1990', 'sports'), ('usa', '?', '?', '?', '?'), ('?', 'honda', '?', '1990', '?'), ('?', 'toyota', '?', '1980', 'economy')}

 S[3]: {('O', 'O', 'O', 'O', 'O')}

 G[4]: {('usa', '?', '?', '?', 'sports'), ('?', '?', 'white', '?', '?'), ('?', 'honda', '?', '?', 'sports'), ('?', '?', '?', '1990', 'sports'), ('usa', '?', '?', '1990', '?'), ('?', 'honda', '?', '1990', '?'), ('?', 'toyota', '?', '1980', 'economy'), ('usa', 'toyota',